# 11\. [**Répartition**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/11_dispatching.ipynb)<br/>([*Dispatching*](https://scikit-learn.org/stable/modules/array_api.html))

**Volume** : 3 pages, 0 exemples, 0 papiers

- ✔ 11.1. [**Prise en charge de l'API Array (expérimental)**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/11_1_array_api.ipynb#array-api-support-experimental)<br/>([_Array API support (experimental)_](https://scikit-learn.org/stable/modules/array_api.html#array-api-support-experimental))
    - ✔ 11.1.1. [**Exemple d'utilisation**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/11_1_array_api.ipynb#example-usage)<br/>([_Example usage_](https://scikit-learn.org/stable/modules/array_api.html#example-usage))
    - ✔ 11.1.2. [**Estimateurs avec prise en charge des entrées compatibles avec l'API Array**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/11_1_array_api.ipynb#estimators-with-support-for-array-api-compatible-inputs)<br/>([_Estimators with support for Array API-compatible inputs_](https://scikit-learn.org/stable/modules/array_api.html#estimators-with-support-for-array-api-compatible-inputs#estimators-with-support-for-array-api-compatible-inputs))
    - ✔ 11.1.3. [**Vérifications communes des estimateurs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/11_1_array_api.ipynb#common-estimator-checks)<br/>([_Common estimator checks_](https://scikit-learn.org/stable/modules/array_api.html#common-estimator-checks))

# 11.1. [**Prise en charge de l'API Array (expérimental)**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/11_1_array_api.ipynb#array-api-support-experimental) ([*Array API support (experimental)*](https://scikit-learn.org/stable/modules/array_api.html#array-api-support-experimental))

La spécification de l'[API Array](https://data-apis.org/array-api/latest/) définit une API standard pour toutes les bibliothèques de manipulation de tableaux avec une API similaire à NumPy.

Certains estimateurs de scikit-learn qui dépendent principalement de NumPy (par opposition à l'utilisation de Cython) pour implémenter la logique algorithmique de leurs méthodes `fit`, `predict` ou `transform` peuvent être configurés pour accepter des structures de données compatibles avec l'API Array et envoyer automatiquement les opérations vers l'espace de noms sous-jacent au lieu de dépendre de NumPy.

À ce stade, cette prise en charge est considérée comme expérimentale et doit être activée explicitement comme expliqué ci-dessous.

> **Remarque** : Actuellement, seules les interfaces `cupy.array_api` et `numpy.array_api` sont connues pour fonctionner avec les estimateurs de scikit-learn.

## <a id='example-usage'></a> 11.1.1. Exemple d'utilisation

Voici un exemple de code pour montrer comment utiliser [CuPy](https://cupy.dev/) pour exécuter [**`LinearDiscriminantAnalysis`**](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis) sur un GPU :

In [5]:
from sklearn.datasets import make_classification
from sklearn import config_context
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import cupy

X_np, y_np = make_classification(random_state=0)
X_cu = cupy.asarray(X_np)
y_cu = cupy.asarray(y_np)
X_cu.device

with config_context(array_api_dispatch=True):
    lda = LinearDiscriminantAnalysis()
    X_trans = lda.fit_transform(X_cu, y_cu)
X_trans.device

ImportError: array_api_compat is required to dispatch arrays using the API specification

Une fois que le modèle est entraîné, les attributs ajustés qui sont des tableaux proviendront également du même espace de noms Array API que les données d'entraînement. Par exemple, si l'espace de noms Array API de CuPy a été utilisé pour l'entraînement, alors les attributs ajustés seront sur le GPU. Nous fournissons une fonction expérimentale `_estimator_with_converted_arrays` qui transfère les attributs d'un estimateur de l'API Array à un ndarray :

In [8]:
from sklearn.utils._array_api import _estimator_with_converted_arrays
cupy_to_ndarray = lambda array: array._array.get()
lda_np = _estimator_with_converted_arrays(lda, cupy_to_ndarray)
X_trans = lda_np.transform(X_np)
type(X_trans)

numpy.ndarray

### <a id='pytorch-support'></a> 11.1.1.1. Prise en charge de PyTorch

Les tenseurs PyTorch sont pris en charge en définissant `array_api_dispatch=True` et en passant les tenseurs directement.

In [6]:
import torch
X_torch = torch.asarray(X_np, device="cuda", dtype=torch.float32)
y_torch = torch.asarray(y_np, device="cuda", dtype=torch.float32)

with config_context(array_api_dispatch=True):
    lda = LinearDiscriminantAnalysis()
    X_trans = lda.fit_transform(X_torch, y_torch)
type(X_trans)
# <class 'torch.Tensor'>
X_trans.device.type

RuntimeError: PyTorch is not linked with support for cuda devices

## <a id='estimators-with-support-for-array-api-compatible-inputs'></a> 11.1.2. Estimateurs prenant en charge les entrées compatibles avec l'API Array

* [**`discriminant_analysis.LinearDiscriminantAnalysis`**](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis) (avec `solver="svd"`)

La couverture pour plus d'estimateurs devrait augmenter avec le temps. Veuillez suivre la [meta-issue dédiée sur GitHub](https://github.com/scikit-learn/scikit-learn/issues/22352) pour suivre les progrès.

## <a id='common-estimator-checks'></a> 11.1.3. Vérifications communes des estimateurs

Ajoutez l'étiquette `array_api_support` à l'ensemble des étiquettes d'un estimateur pour indiquer qu'il prend en charge l'API Array. Cela permettra d'effectuer des vérifications dédiées dans le cadre des tests communs pour vérifier que les résultats des estimateurs sont les mêmes lors de l'utilisation d'entrées NumPy classiques et de l'API Array.

Pour exécuter ces vérifications, vous devez installer [array_api_compat](https://github.com/data-apis/array-api-compat) dans votre environnement de test. Pour exécuter l'ensemble complet des vérifications, vous devez installer à la fois [PyTorch](https://pytorch.org/) et [CuPy](https://cupy.dev/) et disposer d'un GPU. Les vérifications qui ne peuvent pas être exécutées ou qui ont des dépendances manquantes seront automatiquement ignorées.